In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sys
import warnings
from pathlib import Path

module_path = Path().joinpath('../..')
if module_path not in sys.path:
    sys.path.append(module_path)
    
from itertools import product
from more_itertools import chunked
from functools import partial
import multifidelityfunctions as mff
import multiLevelCoSurrogates as mlcs

np.random.seed(20160501)  # Setting seed for reproducibility
OD = mff.forrester

np.set_printoptions(linewidth=200, edgeitems=10, precision=4, suppress=True)
plot_dir = Path('../../plots/')
data_dir = Path('../../files/')

from function_defs import *

# EGO on 2D functions

## 'default' grid of initial points

In [ ]:
low_x = np.linspace(0,1,6).reshape((-1,1))
high_x = low_x[[2,3]].reshape((-1,1))
steps = [.025, .025]
np.random.seed(20160501)

high_xy = np.array(list(product(high_x.flatten(), repeat=2)))
low_xy =  np.array(list(product(low_x.flatten(), repeat=2)))

archive = mlcs.CandidateArchive(ndim=2, fidelities=['high', 'low', 'high-low'])
archive.addcandidates(low_xy, TD_inv.low(low_xy), fidelity='low')
archive.addcandidates(high_xy, TD_inv.high(high_xy), fidelity='high')

mfbo = mlcs.MultiFidelityBO(TD_inv, archive, schema=[2,1])

fig, axes = plt.subplots(6,4, figsize=(16, 20))#, subplot_kw={'projection': '3d'})

idx = 0
for row in chunked(axes.tolist(), 2):
    for ax1, ax2 in zip(*row):
        
        model_surface = mlcs.createsurface(mfbo.models['high'].predict, step=steps, 
                                           u_bound=TD_inv.u_bound, l_bound=TD_inv.l_bound)
        acq = partial(mfbo.utility, gp=mfbo.models['high'], y_max=archive.max['high'])
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            acq_surface = mlcs.createsurface(acq, step=steps, 
                                             u_bound=TD_inv.u_bound, l_bound=TD_inv.l_bound)

        points = [
            mlcs.ScatterPoints(*archive.getcandidates(fidelity='high'), red_dot),
            mlcs.ScatterPoints(*archive.getcandidates(fidelity='low'), blue_circle),
        ]
        surf = mlcs.plotcmaponaxis(ax1, model_surface, title=f'high model - iteration {idx}', point_sets=points)
        fig.colorbar(surf, ax=ax1)
        surf = mlcs.plotcmaponaxis(ax2, acq_surface, title='acquisition function', point_sets=points)
        fig.colorbar(surf, ax=ax2)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mfbo.iteration(idx)
            
        idx += 1
plt.tight_layout()
plt.savefig(f'{plot_dir}2D_BO.pdf')
plt.savefig(f'{plot_dir}2D_BO.png')
plt.show()

## LHS initial sample

In [ ]:
n_low = 12
n_high = 3
np.random.seed(20160501)

low_xy =  low_lhs_sample(ndim=2, nlow=n_low)
high_xy = low_xy[np.random.choice(n_low, n_high, replace=False)]

archive = mlcs.CandidateArchive(ndim=2, fidelities=['high', 'low', 'high-low'])
archive.addcandidates(low_xy, TD_inv.low(low_xy), fidelity='low')
archive.addcandidates(high_xy, TD_inv.high(high_xy), fidelity='high')

mfbo = mlcs.MultiFidelityBO(TD_inv, archive, schema=[2,1])

fig, axes = plt.subplots(8,5, figsize=(16, 20))#, subplot_kw={'projection': '3d'})

idx = 0
for row in chunked(axes.tolist(), 2):
    for ax1, ax2 in zip(*row):
        
        model_surface = mlcs.createsurface(mfbo.models['high'].predict, step=steps, 
                                           u_bound=TD_inv.u_bound, l_bound=TD_inv.l_bound)
        acq = partial(mfbo.utility, gp=mfbo.models['high'], y_max=archive.max['high'])
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            acq_surface = mlcs.createsurface(acq, step=steps, 
                                             u_bound=TD_inv.u_bound, l_bound=TD_inv.l_bound)

        points = [
            mlcs.ScatterPoints(*archive.getcandidates(fidelity='high'), red_dot),
            mlcs.ScatterPoints(*archive.getcandidates(fidelity='low'), blue_circle),
        ]
        surf = mlcs.plotcmaponaxis(ax1, model_surface, title=f'high model - iteration {idx}', point_sets=points)
        fig.colorbar(surf, ax=ax1)
        surf = mlcs.plotcmaponaxis(ax2, acq_surface, title='acquisition function', point_sets=points)
        fig.colorbar(surf, ax=ax2)

        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mfbo.iteration(idx)
            
        idx += 1
plt.tight_layout()
plt.savefig(f'{plot_dir}2D_LHS_BO.pdf')
plt.savefig(f'{plot_dir}2D_LHS_BO.png')
plt.show()